In [1]:
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
import IPython.display as ipd
import librosa
import librosa.display

In [6]:
import pandas as pd
metadata=pd.read_csv("C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_training_part1/labels.csv")
metadata.head(10)

,track,algorithm
0,00050dd7458cf08e594c797930696bce.wav,4
1,00070e7c531000d3dddc735d107275a9.wav,2
2,000f0711027a69b7f3886c2dbcb7d41f.wav,3
3,001e28e66dee24408aaf3480dfb95fbe.wav,1
4,001eee950f60613869544b72cd48fe97.wav,2
5,001f59a6ef884e2071ba1ee52cba8d09.wav,0
6,0022a32237a11ee475c7b96d82369596.wav,3
7,0022e7014db5482004d9e6f731cfbc8b.wav,4
8,00317a6d7effa97c0dc7e1cf1b6d542d.wav,3
9,0036636e89ad53945f0f2707c09f148e.wav,1


In [8]:
metadata_un=pd.read_csv("C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_unseen/labels.csv")
metadata_un.head(10)


,track,algorithm
0,7b0f8df28f98bdcd65e48d4799c1f98f.wav,5
1,211fbbdb342c2bbbac43fd4c908e239a.wav,5
2,807587f93df68fd50120fa1f3684b5ec.wav,5
3,78724d807be41abdd1a4d661d6535d1f.wav,5
4,86195fbf1d21aeeb9a956e56dd88f1b6.wav,5
5,8fa0377cc906aee9ed346cc1b5cb1d3d.wav,5
6,e699913cdd2a6773623b7027ecd32dfe.wav,5
7,a24bbf73e353f430dba5c5ff8d651b6a.wav,5
8,7e05e73230bb5936de69fb5eee9f6762.wav,5
9,5dee2a367819fd0f948b75da0c906ba4.wav,5


In [10]:
def feature_extractor(file):
    data,sample_rate=librosa.load(file,res_type='kaiser_fast')
    mfccs=librosa.feature.mfcc(y=data,sr=sample_rate,n_mfcc=60)    
    mfccs_scaled_features=np.mean(mfccs.T,axis=0)
    return mfccs_scaled_features

In [11]:
import os
audio_dataset_path="C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_training_part1"
audio_dataset_path_un="C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_unseen"

In [12]:
import numpy as np
from tqdm import tqdm
extracted_features=[]
for index,row in tqdm(metadata.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path),str(row["track"]))
    final_class_label=row["algorithm"]
    data=feature_extractor(file_name)
    extracted_features.append([data,final_class_label])


5000it [08:33,  9.74it/s]


In [14]:
for index,row in tqdm(metadata_un.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path_un),str(row["track"]))
    final_class_label=row["algorithm"]
    data=feature_extractor(file_name)
    extracted_features.append([data,final_class_label])

1000it [01:28, 11.29it/s]


In [15]:
import random
random.shuffle(extracted_features)

In [16]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['track','algorithm'])
extracted_features_df

,track,algorithm
0,"[-243.90279, 117.09268, -21.011786, 33.6408, -...",3
1,"[-244.82231, 113.38281, -18.689651, 50.29528, ...",3
2,"[-262.0656, 111.25682, -25.563023, 42.683376, ...",5
3,"[-280.80753, 129.30823, -30.738705, 23.066547,...",2
4,"[-267.67685, 115.20894, -28.6813, 40.30148, -3...",2
...,...,...
5995,"[-241.79272, 104.504364, -35.197643, 47.369072...",1
5996,"[-246.9134, 110.62661, -34.39104, 37.08726, -3...",2
5997,"[-264.63074, 113.37765, -32.592407, 36.436108,...",0
5998,"[-251.85706, 106.08554, -30.47908, 49.70464, -...",1


In [17]:
x=np.array(extracted_features_df['track'].tolist())
y=np.array(extracted_features_df['algorithm'].tolist())

In [18]:
x.shape

(6000, 60)

In [19]:
y.shape

(6000,)

In [20]:
y=np.array(pd.get_dummies(y))

In [21]:
y.shape

(6000, 6)

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [24]:
num_labels=y.shape[1]

In [28]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(60,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [29]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [39]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification_updated.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
139/150 [==========================>...] - ETA: 0s - loss: 0.1344 - accuracy: 0.9496
Epoch 1: val_loss improved from inf to 0.10743, saving model to saved_models\audio_classification_updated.hdf5
150/150 [==============================] - 0s 3ms/step - loss: 0.1386 - accuracy: 0.9483 - val_loss: 0.1074 - val_accuracy: 0.9558
Epoch 2/100
126/150 [========================>.....] - ETA: 0s - loss: 0.1370 - accuracy: 0.9516
Epoch 2: val_loss did not improve from 0.10743
150/150 [==============================] - 0s 2ms/step - loss: 0.1334 - accuracy: 0.9531 - val_loss: 0.1161 - val_accuracy: 0.9550
Epoch 3/100
148/150 [============================>.] - ETA: 0s - loss: 0.1255 - accuracy: 0.9531
Epoch 3: val_loss did not improve from 0.10743
150/150 [==============================] - 0s 3ms/step - loss: 0.1244 - accuracy: 0.9535 - val_loss: 0.1085 - val_accuracy: 0.9633
Epoch 4/100
132/150 [=========================>....] - ETA: 0s - loss: 0.1070 - accuracy: 0.9614
Epoch 4: val_l

In [40]:
test_accuracy=model.evaluate(x_train,y_train,verbose=0)
print(test_accuracy[1])

0.9804166555404663


In [41]:
test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print(test_accuracy[1])

0.9608333110809326


In [42]:
filename="C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_training_part1/00b75d43ebb31451f80e742fdab790d7.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=60)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
prediction_feature=mfccs_scaled_features.reshape(1,-1)

In [43]:
from sklearn.preprocessing import LabelEncoder
predict_x=model.predict(prediction_feature) 
classes_x=np.argmax(predict_x,axis=1)
print(classes_x)

[4]


In [44]:
filename="C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_unseen/0cd4bd4f83b6a4ddb56fb5529460412c.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=60)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
prediction_feature=mfccs_scaled_features.reshape(1,-1)

In [45]:
predict_x=model.predict(prediction_feature) 
classes_x=np.argmax(predict_x,axis=1)
print(classes_x)

[5]


In [46]:
x_train.shape

(4800, 60)

In [47]:
x_test.shape

(1200, 60)

In [52]:
y1=np.array(extracted_features_df['algorithm'].tolist())

In [53]:
x1_train,x1_test,y1_train,y1_test=train_test_split(x,y1,test_size=0.2,random_state=0)

In [54]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [56]:
# fit model no training data
model = XGBClassifier()
model.fit(x1_train, y1_train)

C:\Users\admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:19:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [57]:
# make predictions for test data
y_pred = model.predict(x1_test)
predictions = [round(value) for value in y_pred]

In [58]:
# evaluate predictions
accuracy = accuracy_score(y1_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 96.58%
